In [1]:
import os
import cv2
import numpy as np
import zipfile
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def unzip_file(zip_path, out_path):
  if not os.path.exists(out_path):
    os.makedirs(out_path, exist_ok=True)
  try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
      zip_ref.extractall(out_path)
  except FileNotFoundError:
        print(f"Error: Zip file not found at {zip_path}")
  except zipfile.BadZipFile:
        print(f"Error: Invalid zip file at {zip_path}")
  except Exception as e:
        print(f"An error occurred: {e}")

In [4]:
unzip_file('/content/drive/MyDrive/PS-new.zip','/content/Dataset')

In [5]:
# prompt: resize all images in datset

import os
import cv2
import numpy as np

def resize_images_in_dataset(dataset_path, target_size=(224, 224)):
    for root, _, files in os.walk(dataset_path):
        for file in files:
            if file.endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(root, file)
                try:
                    img = cv2.imread(img_path)
                    if img is not None:
                        resized_img = cv2.resize(img, target_size)
                        cv2.imwrite(img_path, resized_img)
                    else:
                        print(f"Could not read image: {img_path}")
                except Exception as e:
                    print(f"Error processing image {img_path}: {e}")

# Example usage
resize_images_in_dataset('/content/Dataset')


##Dataset Augmentation

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
from PIL import Image
from matplotlib import pyplot as plt

# Define a powerful augmentation configuration
augmentation_generator = ImageDataGenerator(
    rotation_range=40,          # Increased rotation range
    width_shift_range=0.3,     # More horizontal shifting
    height_shift_range=0.3,    # More vertical shifting
    shear_range=0.3,           # Increased shearing
    zoom_range=0.3,            # More zoom variation
    horizontal_flip=True,
    vertical_flip=True,        # Added vertical flipping
    brightness_range=[0.5, 1.5],  # Wider brightness range
    channel_shift_range=50.0,  # Color channel variations
    fill_mode='reflect',       # Better edge handling
    rescale=1./255
)

In [7]:
def generate_augmented_images(image_path, output_dir, num_augmented=10):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Load the image
    try:
        img = Image.open(image_path).convert('RGB') # Convert to RGB to remove alpha channel if present
    except (PIL.UnidentifiedImageError, OSError) as e:
        print(f"Error opening image {image_path}: {e}")
        return
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Get the base filename
    base_filename = os.path.splitext(os.path.basename(image_path))[0]

    # Generate and save augmented images
    i = 0
    for batch in augmentation_generator.flow(img_array, batch_size=1,
                                          save_to_dir=output_dir,
                                          save_prefix=f'aug_{base_filename}',
                                          save_format='jpg'):
        i += 1
        if i >= num_augmented:
            break  # Stop after generating the desired number

In [8]:
!pip install Pillow
from IPython import get_ipython
from IPython.display import display
import PIL

In [9]:
def augment_entire_directory(input_dir, output_dir, num_augmented=10):
    for class_dir in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_dir)
        if os.path.isdir(class_path):
            output_class_dir = os.path.join(output_dir, class_dir)
            os.makedirs(output_class_dir, exist_ok=True)

            for img_file in os.listdir(class_path):
                if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(class_path, img_file)
                    generate_augmented_images(img_path, output_class_dir, num_augmented)

# Bulk augmentation example:
input_directory = '/content/Dataset/PS-new'
output_directory = '/content/Dataset/PS-new'
augment_entire_directory(input_directory, output_directory, num_augmented=12)

In [20]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.applications import EfficientNetB0  # More efficient than ResNet50

# Enhanced Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,  # Increased rotation
    width_shift_range=0.25,
    height_shift_range=0.25,
    shear_range=0.25,
    zoom_range=0.25,
    horizontal_flip=True,
    vertical_flip=True,  # Added vertical flip
    brightness_range=[0.7, 1.3],  # Added brightness adjustment
    fill_mode='nearest',
    validation_split=0.2
)

# Create generators with larger batch size
train_generator = train_datagen.flow_from_directory(
    '/content/Dataset/PS-new',
    target_size=(224, 224),
    batch_size=64,  # Increased batch size
    class_mode='categorical',
    subset='training',
    shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
    '/content/Dataset/PS-new',
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

# Use EfficientNet as base model (better performance than ResNet)
base_model = EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
    pooling=None
)

# Freeze initial layers and gradually unfreeze some
for layer in base_model.layers[:100]:
    layer.trainable = False
for layer in base_model.layers[100:]:
    layer.trainable = True

# Build model with more regularization
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(4, activation='softmax')
])

# Custom learning rate schedule
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.96,
    staircase=True
)

optimizer = Adam(learning_rate=lr_schedule)

# Enhanced callbacks
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    restore_best_weights=True,
    mode='max'
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

model_checkpoint = ModelCheckpoint(
    'best_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

# Train with more epochs
history = model.fit(
    train_generator,
    epochs=100,
    validation_data=validation_generator,
    callbacks=[early_stopping, reduce_lr, model_checkpoint],
    verbose=1
)

# Save model and history
model.save('final_model.keras')
history_df = pd.DataFrame(history.history)
history_df.to_csv('training_history_final.csv', index=False)

Found 3017 images belonging to 4 classes.
Found 752 images belonging to 4 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3074 - loss: 7.7995 - precision_1: 0.3277 - recall_1: 0.2112

48/48 ━━━━━━━━━━━━━━━━━━━━ 135s 2s/step - accuracy: 0.3079 - loss: 7.7929 - precision_1: 0.3283 - recall_1: 0.2111 - val_accuracy: 0.3378 - val_loss: 6.5371 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00 - learning_rate: 0.0010
Epoch 2/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 54s 1s/step - accuracy: 0.3990 - loss: 6.3897 - precision_1: 0.4488 - recall_1: 0.2218 - val_accuracy: 0.3378 - val_loss: 5.3970 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00 - learning_rate: 0.0010
Epoch 3/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - accuracy: 0.4132 - loss: 5.2705 - precision_1: 0.4578 - recall_1: 0.2189 - val_accuracy: 0.3378 - val_loss: 4.4551 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00 - learning_rate: 0.0010
Epoch 4/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - accuracy: 0.4143 - loss: 4.3183 - precision_1: 0.4752 - recall_1: 0.2215 - val_accuracy: 0.3378 - val_loss: 3.6782 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00 - learning_rate: 0.0010
Epoch 5/100
48/

48/48 ━━━━━━━━━━━━━━━━━━━━ 54s 1s/step - accuracy: 0.5124 - loss: 1.9040 - precision_1: 0.6342 - recall_1: 0.3194 - val_accuracy: 0.3910 - val_loss: 1.9445 - val_precision_1: 0.4563 - val_recall_1: 0.1250 - learning_rate: 0.0010
Epoch 10/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 903ms/step - accuracy: 0.5120 - loss: 1.7800 - precision_1: 0.6747 - recall_1: 0.2898

48/48 ━━━━━━━━━━━━━━━━━━━━ 54s 1s/step - accuracy: 0.5120 - loss: 1.7796 - precision_1: 0.6742 - recall_1: 0.2897 - val_accuracy: 0.4309 - val_loss: 1.7740 - val_precision_1: 0.5548 - val_recall_1: 0.1077 - learning_rate: 0.0010
Epoch 11/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - accuracy: 0.5470 - loss: 1.5918 - precision_1: 0.6870 - recall_1: 0.3343 - val_accuracy: 0.3231 - val_loss: 1.7662 - val_precision_1: 0.2783 - val_recall_1: 0.0426 - learning_rate: 0.0010
Epoch 12/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - accuracy: 0.4874 - loss: 1.5736 - precision_1: 0.6346 - recall_1: 0.2817 - val_accuracy: 0.3351 - val_loss: 1.6555 - val_precision_1: 0.7500 - val_recall_1: 0.0040 - learning_rate: 0.0010
Epoch 13/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 82s 1s/step - accuracy: 0.5383 - loss: 1.4262 - precision_1: 0.6741 - recall_1: 0.3245 - val_accuracy: 0.3391 - val_loss: 1.6240 - val_precision_1: 0.2419 - val_recall_1: 0.0199 - learning_rate: 0.0010
Epoch 14/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 54s 1

In [21]:
callbacks = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=15,
        restore_best_weights=True,
        mode='max'
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
]

In [22]:
val_results = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_results[1]*100:.2f}%")

# Lower learning rate for fine-tuning
fine_tune_lr = 0.0001
model.compile(
    optimizer=tf.keras.optimizers.Adam(fine_tune_lr),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tune for a few more epochs
fine_tune_epochs = 10
history_fine = model.fit(
    train_generator,
    epochs=fine_tune_epochs,
    validation_data=validation_generator,
    callbacks=callbacks
)

12/12 ━━━━━━━━━━━━━━━━━━━━ 10s 877ms/step - accuracy: 0.4428 - loss: 1.7661 - precision_1: 0.5616 - recall_1: 0.1077
Validation Accuracy: 42.82%
Epoch 1/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 135s 2s/step - accuracy: 0.5474 - loss: 1.5897 - val_accuracy: 0.3723 - val_loss: 1.7445 - learning_rate: 1.0000e-04
Epoch 2/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - accuracy: 0.5658 - loss: 1.4834 - val_accuracy: 0.3870 - val_loss: 1.7320 - learning_rate: 1.0000e-04
Epoch 3/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - accuracy: 0.5572 - loss: 1.4624 - val_accuracy: 0.3856 - val_loss: 1.7650 - learning_rate: 1.0000e-04
Epoch 4/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - accuracy: 0.5850 - loss: 1.3715 - val_accuracy: 0.3630 - val_loss: 1.6946 - learning_rate: 1.0000e-04
Epoch 5/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - accuracy: 0.5770 - loss: 1.3551 - val_accuracy: 0.3604 - val_loss: 1.7451 - learning_rate: 1.0000e-04
Epoch 6/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - accuracy: 0.6075 - loss: 1.3067 

In [23]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,817,385 (48.89 MB)

 Trainable params: 4,202,464 (16.03 MB)

 Non-trainable params: 209,991 (820.28 KB)

 Optimizer params: 8,404,930 (32.06 MB)

##Accuracy Visualization

In [24]:
epochs = list(range(1,100)) # Change epochs to a list
plt.plot(epochs,training_history.history['accuracy'],color='red',label='Training Accuracy')
plt.plot(epochs,training_history.history['val_accuracy'],color='blue',label='Validation Accuracy')
plt.xlabel('No. of Epochs')
plt.ylabel('Accuracy Result')
plt.title('Vissualization of Accuracy Result')
plt.legend()
plt.show()

NameError: name 'training_history' is not defined

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
plt.figure(figsize=(40,40))
sns.heatmap(cm,annot=True,annot_kws={'size':10})
plt.xlabel('Predicted Class',fontsize=20)
plt.ylabel('Actual Class',fontsize=20)
plt.title('Plant Disease Detection Confusion Matrix',fontsize=30)
plt.show()